## <center>POKEMON DCGAN</center>

#### Importing all the required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, metrics, losses, optimizers, Model
import cv2
import os
tf.random.set_seed(45)

#### Model

In [ ]:
class DCGANGenerator(Model):
    def __init__(self, latent_dim):
        super(DCGANGenerator, self).__init__()
        
        self.dense_1 = layers.Dense(units=latent_dim*2*2, input_dim=latent_dim)
        self.activation_1 = layers.Activation('relu')
        self.reshape_1 = layers.Reshape(target_shape=(2, 2, latent_dim))

        self.upsampling_2 = layers.UpSampling2D()
        self.conv_2 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_2 = layers.BatchNormalization(momentum=0.7)
        self.activation_2 = layers.Activation('relu')

        self.upsampling_3 = layers.UpSampling2D()
        self.conv_3 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_3 = layers.BatchNormalization(momentum=0.7)
        self.activation_3 = layers.Activation('relu')

        self.upsampling_4 = layers.UpSampling2D()
        self.conv_4 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_4 = layers.BatchNormalization(momentum=0.7)
        self.activation_4 = layers.Activation('relu')

        self.upsampling_5 = layers.UpSampling2D()
        self.conv_5 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_5 = layers.BatchNormalization(momentum=0.7)
        self.activation_5 = layers.Activation('relu')

        self.upsampling_6 = layers.UpSampling2D()
        self.conv_6 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_6 = layers.BatchNormalization(momentum=0.8)
        self.activation_6 = layers.Activation('relu')

        self.upsampling_7 = layers.UpSampling2D()
        self.conv_7 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_7 = layers.BatchNormalization(momentum=0.7)
        self.activation_7 = layers.Activation('relu')

        self.conv_8 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_8 = layers.BatchNormalization(momentum=0.7)
        self.activation_8 = layers.Activation('relu')

        self.conv_9 = layers.Conv2D(filters=3, kernel_size=(3, 3), strides=1, padding='same')
        self.activation_9 = layers.Activation('tanh')
    
    def call(self, x):

        x = self.dense_1(x)
        x = self.activation_1(x)
        x = self.reshape_1(x)

        x = self.upsampling_2(x)
        x = self.conv_2(x)
        x = self.norm_2(x)
        x = self.activation_2(x)

        x = self.upsampling_3(x)
        x = self.conv_3(x)
        x = self.norm_3(x)
        x = self.activation_3(x)

        x = self.upsampling_4(x)
        x = self.conv_4(x)
        x = self.norm_4(x)
        x = self.activation_4(x)

        x = self.upsampling_5(x)
        x = self.conv_5(x)
        x = self.norm_5(x)
        x = self.activation_5(x)

        x = self.upsampling_6(x)
        x = self.conv_6(x)
        x = self.norm_6(x)
        x = self.activation_6(x)

        x = self.upsampling_7(x)
        x = self.conv_7(x)
        x = self.norm_7(x)
        x = self.activation_7(x)

#         x = self.upsampling_8(x)
        x = self.conv_8(x)
        x = self.norm_8(x)
        x = self.activation_8(x)

        x = self.conv_9(x)
        out = self.activation_9(x)

        return out

In [ ]:
class DCGANDiscriminator(Model):
    def __init__(self, image_shape):
        super(DCGANDiscriminator, self).__init__()

        self.conv_1 = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2, padding='same')
        self.activation_1 = layers.LeakyReLU(alpha=0.2)
        self.dropout_1 = layers.Dropout(rate=0.25)

        self.conv_2 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2, padding='same')
        self.zeropad_2 = layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
        self.norm_2 = layers.BatchNormalization(momentum=0.8)
        self.activation_2 = layers.LeakyReLU(alpha=0.2)
        self.dropout_2 = layers.Dropout(rate=0.25)

        self.conv_3 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2, padding='same')
        self.norm_3 = layers.BatchNormalization(momentum=0.8)
        self.activation_3 = layers.LeakyReLU(alpha=0.1)
        self.dropout_3 = layers.Dropout(rate=0.25)

        self.conv_4 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_4 = layers.BatchNormalization(momentum=0.8)
        self.activation_4 = layers.LeakyReLU(alpha=0.1)
        self.dropout_4 = layers.Dropout(rate=0.25)

        self.flatten = layers.Flatten()
        self.dense_5 = layers.Dense(units=1)
        self.activation_5 = layers.Activation('sigmoid')
    
    def call(self, x):
        # print(X.shape)
        x = self.conv_1(x)
        x = self.activation_1(x)
        x = self.dropout_1(x)
        # print(x.shape)

        x = self.conv_2(x)
        x = self.zeropad_2(x)
        x = self.norm_2(x)
        x = self.activation_2(x)
        x = self.dropout_2(x)
        # print(x.shape)

        x = self.conv_3(x)
        x = self.norm_3(x)
        x = self.activation_3(x)
        x = self.dropout_3(x)
        # print(x.shape)

        x = self.conv_4(x)
        x = self.norm_4(x)
        x = self.activation_4(x)
        x = self.dropout_4(x)
        # print(x.shape)

        x = self.flatten(x)
        # print(x.shape)

        x = self.dense_5(x)
        # print(x.shape)
        out = self.activation_5(x)

        return out

In [ ]:
def train_step(gen_model, dis_model, loss_method, optimizer_method_gen, optimizer_method_dis, batch_size, latent_dim, X):
    latent_vector = tf.random.normal(shape=(batch_size, latent_dim))
    valid_labels = tf.ones(shape=(batch_size, 1))
    fake_labels = tf.zeros(shape=(batch_size, 1))
    
    with tf.GradientTape() as tape:
        gen_out = gen_model(latent_vector, training=True)
        real_out = dis_model(X, training=True)
        fake_out = dis_model(gen_out, training=True)
        real_loss = loss_method(valid_labels, real_out)
        fake_loss = loss_method(fake_labels, fake_out)
        dis_loss = tf.multiply(tf.add(real_loss, fake_loss), 1.0)
    variables = gen_model.trainable_variables + dis_model.trainable_variables
    gradients = tape.gradient(dis_loss, variables)
    optimizer_method_dis.apply_gradients(zip(gradients, variables))

    with tf.GradientTape() as tape:
        gen_out = gen_model(latent_vector, training=True)
        fake_out = dis_model(gen_out, training=True)
        gen_loss = loss_method(valid_labels, fake_out)
    variables = gen_model.trainable_variables
    gradients = tape.gradient(gen_loss, variables)
    optimizer_method_gen.apply_gradients(zip(gradients, variables))

    return gen_loss, dis_loss

In [ ]:
def generate_images(epoch, latent_dim, gen_model):
    r, c = 5, 5
    latent_vector = tf.random.normal(shape=(r*c, latent_dim))
    gen_imgs = gen_model(latent_vector, training=True)
    gen_imgs = gen_imgs * 0.5 + 0.5
    fig, axis = plt.subplots(r, c, figsize=(15, 15))
    idx = 0
    for i in range(r):
        for j in range(c):
            axis[i, j].imshow(gen_imgs[idx])
            # axis[i, j].imshow(gen_imgs[idx, :, :, 0], cmap='gray')
            axis[i, j].axis('off')
            idx+=1
    fig.savefig('data/dcgan_color/output/{0}.png'.format(epoch))
    plt.close()
# generate_images(epoch, latent_dim, gen_model)

#### Dataset Prepration

In [ ]:
# This list contains all the image data
data = []
image_shape = (128, 128, 3)
# image_shape = (128, 128)
latent_dim = 128
batch_size = 16
data_dir = 'data/dcgan_color/pokemon/'
ckpt_dir = 'data/dcgan_color/checkpoint/'

for file_name in os.listdir(path=data_dir):
    img_path = os.path.join(data_dir, file_name)
    img = cv2.imread(img_path, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if img.shape[0]<=120 or img.shape[1]<=120:
        img = np.pad(img, pad_width=((4, 4), (4, 4), (0, 0)), mode='constant')
    else:
        img = cv2.resize(img, (128, 128))
#     cv2.imshow('Image', img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data.append(img)
data = np.array(data)
print(data.shape)

In [ ]:
# Here we are creating training batch
train_batch = tf.data.Dataset.from_tensor_slices(data).shuffle(1000).batch(batch_size, drop_remainder=True)

In [ ]:
gen_model = DCGANGenerator(latent_dim=latent_dim)
dis_model = DCGANDiscriminator(image_shape=image_shape)

In [ ]:
loss_method = losses.BinaryCrossentropy()
optimizer_method_gen = optimizers.Adam(learning_rate=0.001)
optimizer_method_dis = optimizers.Adam(learning_rate=0.0001)
ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer_g=optimizer_method_gen, optimizer_d=optimizer_method_dis, g_model=gen_model, d_model=dis_model)
manager = tf.train.CheckpointManager(checkpoint=ckpt, directory=ckpt_dir, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print('Restored from last checkpoint : {0}'.format(int(ckpt.step)))
START_EPOCH = int(ckpt.step)
EPOCHS = 10000

In [ ]:
for epoch in range(START_EPOCH, EPOCHS):
    gen_loss = metrics.Mean()
    dis_loss = metrics.Mean()
    for X in train_batch:
        X = (tf.cast(X, dtype='float32')-127.5)/127.5
        g_loss, d_loss = train_step(gen_model, dis_model, loss_method, optimizer_method_gen, optimizer_method_dis, batch_size, latent_dim, X)
        gen_loss.update_state(g_loss)
        dis_loss.update_state(d_loss)
    ckpt.step.assign_add(1)
    manager.save()
    print('Epoch : {0}\tGen Loss : {1}\tDis Loss : {2}'.format(epoch, gen_loss.result(), dis_loss.result()))
    if epoch%2 == 0:
        generate_images(epoch, latent_dim, gen_model)